<a href="https://colab.research.google.com/github/shreyasr-upenn/asr-error-correction-cis522/blob/main/Finetune_GPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS 522 - Final Project
### Team: Transcriptionists
Members:
Manni Arora - manni@seas.upenn.edu
Pooja Dattatri - poojadat@seas.upenn.edu
Shreyas Ramesh - shreyasr@seas.upenn.edu

### Install dependencies


In [ ]:
!pip install wandb
!pip install openai
!pip install jiwer

     |████████████████████████████████| 1.8 MB 4.6 MB/s 
     |████████████████████████████████| 181 kB 39.0 MB/s 
     |████████████████████████████████| 144 kB 38.7 MB/s 
     |████████████████████████████████| 63 kB 996 kB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=caac3265bd39127e09059d0f00ac5f923fb0449e69b5d02902dc75ba54be30b8
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
     |████████████████████████████████| 42 kB 837 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 162 kB 8.6 MB/s 
  Created wheel for openai: filename=openai-0.18.1-py3-none-any.whl size=53168 sha256=e88b0e0134862a6792c3e15a574a01e06b91fea40c500911e6e8512946810560
  Stored in directory: /root/.cache/pip/wheels/5a/bf/24/fcdc9d2b81f9c7e565bb2036ec9f7cc930

In [ ]:
import sys
import os
import json
from collections import defaultdict
import csv
import openai
import random
from jiwer import wer
openai.api_key = ""
os.environ['OPENAI_API_KEY']=openai.api_key

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Prepare training data

In [ ]:
with open('/content/drive/Shareddrives/CIS522Project/Data/output_ami_pretrained/fine_tune_data.jsonl') as f:
  content = f.readlines()
  random.shuffle(content)
  file_lst = open('train.jsonl', 'w+')
  for i in range(0, len(content)):
    file_lst.write(content[i])
  file_lst.close()

In [ ]:
jsonl_filename = 'train.jsonl'

In [ ]:
!head '{jsonl_filename}'
!wc -lw '{jsonl_filename}'

{"prompt": "ASR output: okay\n", "completion": "ASR correction: okay\n###\n"}
{"prompt": "ASR output: okay\n", "completion": "ASR correction: okay\n###\n"}
{"prompt": "ASR output: co\n", "completion": "ASR correction: cool\n###\n"}
{"prompt": "ASR output: yeah\n", "completion": "ASR correction: yeah\n###\n"}
{"prompt": "ASR output: oh okay oka kay\n", "completion": "ASR correction: oh okay okay okay\n###\n"}
{"prompt": "ASR output: so i think maybe we can agree on on a lounge in a separate room if we can manage to to find space elsewhere\n", "completion": "ASR correction: okay so i think maybe we can agree on on the lounge in a separate room if we can manage to to find space elsewhere\n###\n"}
{"prompt": "ASR output: i think if we go with the um design plate thing we'll have to\n", "completion": "ASR correction: i think if we go with the um design plate thing we'll have to\n###\n"}
{"prompt": "ASR output: or is there no problem\n", "completion": "ASR correction: or w is there no proble

### Finetune on OpenAI's GPT-3 babbage 

In [ ]:
!openai api fine_tunes.create -t '{jsonl_filename}' -m babbage

Upload progress: 100% 1.41M/1.41M [00:00<00:00, 1.77Git/s]
Uploaded file from train.jsonl: file-nugnFydaaa5fb2fG226xL9nz
Created fine-tune: ft-jDD5OERU8Tz6GxKYEt8wLF64
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-04-20 21:59:03] Created fine-tune: ft-jDD5OERU8Tz6GxKYEt8wLF64
[2022-04-20 21:59:21] Fine-tune costs $0.67
[2022-04-20 21:59:22] Fine-tune enqueued. Queue number: 0
[2022-04-20 21:59:27] Fine-tune started
[2022-04-20 22:03:47] Completed epoch 1/4
[2022-04-20 22:07:44] Completed epoch 2/4
[2022-04-20 22:11:42] Completed epoch 3/4

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-jDD5OERU8Tz6GxKYEt8wLF64



In [ ]:
!openai api fine_tunes.follow -i ft-jDD5OERU8Tz6GxKYEt8wLF64

[2022-04-20 21:59:03] Created fine-tune: ft-jDD5OERU8Tz6GxKYEt8wLF64
[2022-04-20 21:59:21] Fine-tune costs $0.67
[2022-04-20 21:59:22] Fine-tune enqueued. Queue number: 0
[2022-04-20 21:59:27] Fine-tune started
[2022-04-20 22:03:47] Completed epoch 1/4
[2022-04-20 22:07:44] Completed epoch 2/4
[2022-04-20 22:11:42] Completed epoch 3/4
[2022-04-20 22:15:41] Completed epoch 4/4
[2022-04-20 22:15:59] Uploaded model: babbage:ft-cis-700-33-2022-04-20-22-15-58
[2022-04-20 22:16:02] Uploaded result file: file-ZR674Wb6H4rX7vEUqbrPZHo2
[2022-04-20 22:16:03] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m babbage:ft-cis-700-33-2022-04-20-22-15-58 -p <YOUR_PROMPT>


### Evaluate model

In [ ]:
def get_label(prompt, finetuned_model):
  response = openai.Completion.create(
      model=finetuned_model,
      prompt="ASR output: {prompt}\nASR correction: ".format(
          prompt=prompt
      ),
      temperature=0.7,
      max_tokens=64,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["###"]
      )
  output =  response['choices'][0]['text']
  return output

In [ ]:
outputs = []
import pandas as pd    

with open('/content/drive/Shareddrives/CIS522Project/Data/output_ami_pretrained/test_data.jsonl') as f:
    lines = f.readlines()
    print(len(lines))
    output = []
    true_labels = []
    for line in lines:
      prompt = line.split('", "completion": "')[0].split("{\"prompt\": \"")[1].replace('"','')
      completion = line.split('", "completion": "')[1][:-6]
      if not completion:
        continue
      gpt3_label = get_label(prompt, 'babbage:ft-cis-700-33-2022-04-20-22-15-58')
      if not gpt3_label:
        continue
      true_labels.append(completion.split("ASR correction: ")[1])
      output.append(gpt3_label)
      print(f'[ASR output] {prompt}')
      print(f'[Suggested correction] {gpt3_label}')
      print('~'*100)

21942


### WER

In [ ]:
print(round(wer(output, true_labels)*100, 2), "%")

27.14 %
